In [1]:
import jax.numpy as jnp
import jax.scipy.linalg as jla
import jax
import numpy as np
import matplotlib.pyplot as plot
import dynamiqs as dq

In [8]:
def com(A,B):
    return A@B + B@A

In [9]:
N = 10
X = (dq.create(N) + dq.destroy(N))/jnp.sqrt(2.)
P = (dq.create(N) - dq.destroy(N))/1j/jnp.sqrt(2.)
I = dq.eye(N)
Xz = dq.tensor(X,dq.sigmaz())
alpha = 2.0
Ix = dq.tensor(I,dq.sigmax())
beta = 1.4
UXz = jla.expm(-1j*alpha*Xz)
UIx = jla.expm(-1j*beta*Ix)
A = UXz@UIx
B = jla.expm(-1j*(alpha*Xz + beta*Ix))

In [27]:
import sympy as sp
from sympy.physics.quantum import TensorProduct

alpha, beta = sp.symbols('alpha beta')
# Define Pauli matrices
sigma_x = sp.Matrix([[0, 1], [1, 0]])
sigma_z = sp.Matrix([[1, 0], [0, -1]])
I2 = sp.Matrix([[1,0],[0,1]])

# Define the tensor products
Ix = alpha*TensorProduct(I2,sigma_x)
Xz = beta*TensorProduct(sigma_x, sigma_z)

In [36]:
def BCH(A,B,n=3):
    total = A + B

    comAB = com(A,B)
    total += 1/2 * comAB

    comAAB = com(A,comAB)
    comBAB = com(B,comAB)
    total += 1/12*(comAAB + comBAB)

    comBAAB = com(B,comAAB)
    print(comBAAB)
    total += 1/24 * comBAAB

    print( com(A,com(A,com(A,com(A,B)))) + com(B,com(B,com(B,com(B,A))))  )
    return total

In [37]:
BCH(Ix,Xz)

Matrix([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])
Matrix([[0, 16*alpha*beta**4, 16*alpha**4*beta, 0], [16*alpha*beta**4, 0, 0, -16*alpha**4*beta], [16*alpha**4*beta, 0, 0, 16*alpha*beta**4], [0, -16*alpha**4*beta, 16*alpha*beta**4, 0]])


Matrix([
[                                       0, -0.333333333333333*alpha*beta**2 + alpha,   0.333333333333333*alpha**2*beta + beta,                          -1.0*alpha*beta],
[-0.333333333333333*alpha*beta**2 + alpha,                                        0,                           1.0*alpha*beta,  -0.333333333333333*alpha**2*beta - beta],
[  0.333333333333333*alpha**2*beta + beta,                          -1.0*alpha*beta,                                        0, -0.333333333333333*alpha*beta**2 + alpha],
[                          1.0*alpha*beta,  -0.333333333333333*alpha**2*beta - beta, -0.333333333333333*alpha*beta**2 + alpha,                                        0]])